In [4]:
import pandas as pd
pd.options.mode.chained_assignment = None
import requests
import re
from bs4 import BeautifulSoup
import json

In [1]:
def findQuantity(text):
    unit="(g|kg|l|ml| litro)"
    x = re.search("[0-9,]+"+unit, text, re.IGNORECASE)
    if x == None:
        return "";
    return x.group()


In [3]:
def findRelPrice(price, quantity):
    if quantity=="":
        return "";
    qt = int(''.join(filter(str.isdigit, quantity)))
    unit = ''.join(filter(str.isalpha, quantity))
    unit = unit.lower()
    
    relPrice = float(price)/qt

    if unit == 'g' or unit == 'ml':
        relPrice*=1000
    
    return str(round(relPrice, 2)) +  ' R$/' + ("kg" if (unit == 'g' or unit == 'kg') else 'L')

In [3]:
baseURL = "https://paodeacucar.resultspage.com/search?p=Q&ts=json-full&lot=json&w={}&cnt=12&ref=www.paodeacucar.com&ua=Mozilla%2F5.0%20(Windows%20NT%2010.0%3B%20Win64%3B%20x64)%20AppleWebKit%2F537.36%20(KHTML%2C%20like%20Gecko)%20Chrome%2F81.0.4044.138%20Safari%2F537.36&ep.selected_store=501"

In [22]:
def fetchItems(products):
    results = pd.DataFrame(columns=['searchKey', 'productName', 'price', 'quantity', 'relPrice'])

    for key in products:
#         print(key)
        url = baseURL.format(key.lower().replace(' ', '+'))
        html = requests.get(url).text
        soup = BeautifulSoup(html, "html.parser")
        items = json.loads(str(soup))['results']
        for i, item in enumerate(items):
            qt = findQuantity(item['title'])
            if 'price' not in item:
                continue
            price = float(item['price'].replace(',','.'))
            if int(item['instock']) > 0:
                results.loc[key.lower().replace(' ', '_') + '_' + str(i)] = [key, item['title'], price, qt, findRelPrice(price, qt)]

    return results

In [23]:
def getSuggested(results):
    suggested = results.drop_duplicates(subset=['searchKey'], keep='first')
    return suggested

In [7]:
def getMinPrice(results):
    min_price = results.groupby('searchKey').min()
    return min_price

In [8]:
def calculatePrices(chosen):
    chosen['amount'] = 0
    print("Quantos de cada um dos itens abaixo você deseja comprar?")
    for i,item in chosen.iterrows(): 
        print(item['productName'] + " (R$ " + str(item['price']) + ")")
        chosen.loc[i,'amount'] = int(input())

    chosen['totalPrice'] = chosen.amount*chosen.price
    return chosen

In [8]:
key = "Maca Gala"

url = baseURL.format(key.lower().replace(' ', '+'))
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")
items = json.loads(str(soup))['results']

In [9]:
items

[{'title': 'Macarrão Instantâneo de Galinha Suave Nissin Miojo Turma da Mônica Pacote 85g',
  'url': 'https://www.paodeacucar.com/produto/42350',
  'text': '<b>MAC</b> INST NISSIN LAMEN SU <b>GAL</b> 85G',
  'rank': 1,
  'shelfList': [{'id': 4326, 'name': 'Massas-Instantâneas', 'parentId': 4225},
   {'id': 4225, 'name': 'Massas-Instantâneas', 'parentId': None}],
  's_id': 26158,
  'image_url': 'http://www.paodeacucar.com/img/uploads/1/832/604832x200x200.png',
  'price': '1,69',
  'instock': '1',
  'shelfname': 'massas-instantâneas',
  'sku': '0791540',
  'mainShelfId': '4326',
  'brand': 'NISSIN',
  'categorias': ['Massas Instantâneas'],
  'suggestions': [{'rank': '1',
    'phrase': 'Black-friday-2019_massasinstantaneas'},
   {'rank': '2', 'phrase': 'Acaoprecosoff_03_massas'}]},
 {'title': 'Maçã Gala Bandeja 500g',
  'url': 'https://www.paodeacucar.com/produto/78635',
  'text': '<b>Maçã</b> <b>Gala</b> Bandeja 500g',
  'rank': 2,
  'shelfList': [{'id': 4252, 'name': 'Frutas', 'parentId

In [3]:
baseURL = "https://paodeacucar.resultspage.com/search?p=Q&ts=json-full&lot=json&w={}&cnt=12&ref=www.paodeacucar.com&ua=Mozilla%2F5.0%20(Windows%20NT%2010.0%3B%20Win64%3B%20x64)%20AppleWebKit%2F537.36%20(KHTML%2C%20like%20Gecko)%20Chrome%2F81.0.4044.138%20Safari%2F537.36&ep.selected_store=501"

In [24]:
key = 'Doritos'

In [25]:
url = baseURL.format(key.lower().replace(' ', '+'))
html = requests.get(url).text
soup = BeautifulSoup(html, "html.parser")
items = json.loads(str(soup))['results']

In [26]:
items

[{'title': 'Salgadinho de Milho Queijo Nacho Elma Chips Doritos Pacote 30g',
  'url': 'https://www.paodeacucar.com/produto/456454',
  'text': 'SALG <b>DORITOS</b> NACHO 30G',
  'rank': 1,
  'shelfList': [{'id': 4349, 'name': 'Salgadinho-e-Snacks', 'parentId': 4228},
   {'id': 4228, 'name': 'Salgadinho-e-Snacks', 'parentId': None}],
  's_id': 0,
  'image_url': 'http://www.paodeacucar.com/img/uploads/1/803/645803x200x200.jpg',
  'price': '2,99',
  'instock': '1',
  'shelfname': 'salgadinho-e-snacks',
  'sku': '1221149',
  'mainShelfId': '4349',
  'categorias': ['Salgadinho e Snacks']},
 {'title': 'Salgadinho de Milho Original Doritos Dippas Pacote 96g',
  'url': 'https://www.paodeacucar.com/produto/411670',
  'text': '<b>DORITOS</b> DIPPAS 96G',
  'rank': 2,
  'shelfList': [{'id': 4349, 'name': 'Salgadinho-e-Snacks', 'parentId': 4228},
   {'id': 4228, 'name': 'Salgadinho-e-Snacks', 'parentId': None}],
  's_id': 25140,
  'image_url': 'http://www.paodeacucar.com/img/uploads/1/139/665139x20